<a href="https://colab.research.google.com/github/ABHINAV-GOPINADH/deep_Learning/blob/main/spoiler_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d rmisra/imdb-spoiler-dataset

Dataset URL: https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 90% 299M/331M [00:05<00:01, 26.1MB/s]
100% 331M/331M [00:05<00:00, 60.6MB/s]


In [ ]:
!unzip imdb-spoiler-dataset.zip

Archive:  imdb-spoiler-dataset.zip
replace IMDB_movie_details.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace IMDB_reviews.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import json
import pandas as pd

# Load JSON lines (one record per line)
with open("IMDB_movie_details.json", "r") as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
df = pd.DataFrame(data)
df.head()


,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...


In [ ]:
import nltk
nltk.download('punkt')  # ✅ Fixes LookupError
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

sentence_nodes = []

for _, row in df.iterrows():
    movie_id = row['movie_id']

    # Non-spoiler sentences (label = 0)
    if pd.notna(row['plot_summary']):
        for s in sent_tokenize(row['plot_summary']):
            sentence_nodes.append({
                'sentence': s.strip(),
                'spoiler': 0,
                'movie_id': movie_id
            })

    # Spoiler sentences (label = 1)
    if pd.notna(row['plot_synopsis']):
        for s in sent_tokenize(row['plot_synopsis']):
            sentence_nodes.append({
                'sentence': s.strip(),
                'spoiler': 1,
                'movie_id': movie_id
            })

import pandas as pd
sent_df = pd.DataFrame(sentence_nodes)
sent_df = sent_df.drop_duplicates(subset=['sentence'])  # Optional
sent_df.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,sentence,spoiler,movie_id
0,"Former CIA analyst, Jack Ryan is in England wi...",0,tt0105112
1,It is revealed that some people are trying to ...,0,tt0105112
2,"Afterwards, he learns that they're Irish revol...",0,tt0105112
3,During his court hearing the one that's still ...,0,tt0105112
4,"However, whilst the man is being transported, ...",0,tt0105112


In [ ]:
value_counts = sent_df['spoiler'].value_counts()
print(value_counts)

spoiler
1    103337
0      7772
Name: count, dtype: int64


In [ ]:
sent_df.isnull().sum()

,0
sentence,0
spoiler,0
movie_id,0


In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from tqdm import tqdm

# Load pre-trained BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
model.eval()  # Disable training mode


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# Define Function to Convert a Sentence to an Embedding
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Use [CLS] token


In [ ]:
#Generate Embeddings for All Sentences
embeddings = []

for sentence in tqdm(sent_df['sentence']):
    try:
        vec = get_bert_embedding(sentence)
    except:
        vec = np.zeros(768)  # fallback if BERT fails
    embeddings.append(vec)

sent_df['embedding'] = embeddings


100%|██████████| 111109/111109 [5:30:39<00:00,  5.60it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Create the directory if it doesn't exist
save_dir = '/content/drive/MyDrive/spoiler_detection'
os.makedirs(save_dir, exist_ok=True)


In [ ]:
sent_df.to_pickle(os.path.join(save_dir, 'sent_df_with_embeddings.pkl'))


NameError: name 'sent_df' is not defined

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

sent_df = pd.read_pickle('/content/drive/MyDrive/spoiler_detection/sent_df_with_embeddings.pkl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(sent_df)

                                                 sentence  spoiler  \
0       Former CIA analyst, Jack Ryan is in England wi...        0   
1       It is revealed that some people are trying to ...        0   
2       Afterwards, he learns that they're Irish revol...        0   
3       During his court hearing the one that's still ...        0   
4       However, whilst the man is being transported, ...        0   
...                                                   ...      ...   
112288  However, David's problems are exacerbated by a...        0   
112289  Karen, a beautiful, ambitious, ruthless, senio...        0   
112290  To further her aims she hires the services of ...        0   
112291  At the date of the merger draws near and all t...        0   
112292          Written by\nMark Smith <msmith@osi.co.uk>        0   

          movie_id                                          embedding  
0        tt0105112  [-0.717707, 0.046108123, -0.54524463, 0.175978...  
1        tt0105

In [ ]:
import torch

edge_list = []

# Group by movie and connect sentences in sequence
grouped = sent_df.groupby("movie_id")
for _, group in grouped:
    indices = group.index.tolist()
    for i in range(len(indices) - 1):
        edge_list.append([indices[i], indices[i+1]])
        edge_list.append([indices[i+1], indices[i]])  # Undirected

# Convert to PyTorch tensor
edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

# Sanity check
print("edge_index shape:", edge_index.shape)
print("Max index:", edge_index.max().item(), "  |  Node count:", len(sent_df))


edge_index shape: torch.Size([2, 219074])
Max index: 111108   |  Node count: 111109


In [ ]:
# Prepare Node Features and Labels

import numpy as np

x = torch.tensor(np.stack(sent_df['embedding'].values), dtype=torch.float)
y = torch.tensor(sent_df['spoiler'].values, dtype=torch.long)

from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)



In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.8 MB/s eta 0:00:00


In [ ]:
#Create PyTorch Geometric Data Object

from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index, y=y)
print(data)


Data(x=[111109, 768], edge_index=[2, 219074], y=[111109])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(768, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2)  # binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
sent_df = sent_df.reset_index(drop=True)


In [ ]:
print("sent_df length:", len(sent_df))
print("Max index in sent_df:", sent_df.index.max())


sent_df length: 111109
Max index in sent_df: 111108


In [ ]:
print("Total embeddings:", len(sent_df['embedding']))
print("Shape of one embedding:", sent_df['embedding'].iloc[0].shape)


Total embeddings: 111109
Shape of one embedding: (768,)


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(768, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(1, 51):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 10, Loss: 0.3379
Epoch 20, Loss: 0.1747
Epoch 30, Loss: 0.1467
Epoch 40, Loss: 0.1284
Epoch 50, Loss: 0.1215


✅ What You’ve Achieved So Far:


✅ Data preprocessing	Done

✅ BERT embeddings	Loaded

✅ Graph construction	Using chain edges per movie (efficient)

✅ GCN training	50 epochs done — loss decreasing steadily ✔️

✅ No crashes or OOM	Confirmed

📉 Loss is going down:

Epoch 10 → 0.3379

Epoch 50 → 0.1215

This is a strong sign that the model is learning to classify spoilers vs. non-spoilers effectively.



In [ ]:
model.eval()
_, pred = model(data).max(dim=1)

correct = (pred == data.y).sum().item()
accuracy = correct / data.num_nodes
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9413


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(data.y.cpu(), pred.cpu(), target_names=["Non-Spoiler", "Spoiler"]))


              precision    recall  f1-score   support

 Non-Spoiler       0.88      0.19      0.31      7772
     Spoiler       0.94      1.00      0.97    103337

    accuracy                           0.94    111109
   macro avg       0.91      0.59      0.64    111109
weighted avg       0.94      0.94      0.92    111109



In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/spoiler_detection/spoiler_gnn_model.pt")


In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)  # shape: [111109, 2]
